In [1]:
import pandas as pd
import numpy as np
import tarfile
import io
import glob
import dask.dataframe as dd

# import xgboost as xgb
# from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

import lightgbm as lgb
import warnings
warnings.simplefilter("ignore")



tar = tarfile.open('../data/raw/20181120_track_features.tar.gz', 'r:gz')
csv_files = tar.getnames()

tf_df_list = []

for csv_file in [csv_files[2], csv_files[4]]:
    csv_contents = tar.extractfile(csv_file).read()
    tf_df_list.append(pd.read_csv(io.BytesIO(csv_contents), encoding='utf8'))

tf_df = pd.concat(tf_df_list, ignore_index=True)
tf_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)

kmean300_df = pd.read_csv('../data/interim/all_data/mbKMeans300clusters.csv', usecols=['track_id','clus'])
kmean300_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)



In [2]:
def get_sim(df_hist, df_lookup, sim_file_list, score_name_list):
    df_hist['ListenYes'] = (df_hist['skip_2'] == False)*1
    df_hist['ListenYes'].replace(0, -1, inplace = True)
    df_hist = df_hist.groupby(['session_id', 'clus']).agg({'ListenYes':['sum']})
    df_hist = df_hist.reset_index()
    df_hist.columns = df_hist.columns.droplevel(level = 1) # take out the unwanted level
    df_pivot = pd.pivot_table(df_hist, values = 'ListenYes',index='session_id', columns='clus')
    df_pivot = df_pivot.fillna(0)
    
    
    for sim_file, score_name in zip(sim_file_list, score_name_list):
        sim_matrix = pd.read_csv(sim_file).drop(columns=['Unnamed: 0'])
        sim_matrix.columns = list(map(str, range(0,len(sim_matrix))))
        df_sim_session = df_pivot.dot(sim_matrix)/sim_matrix.sum()
        
        df_lookup[score_name] = df_sim_session.lookup(df_lookup['session_id'],df_lookup['clus'].astype(str))
    
    return df_lookup

In [3]:
file_list = []
for n in range(9):
    file_list = file_list + glob.glob('../data/raw/training_set/log_'+str(n)+'*.csv')

import random
random.Random(23).shuffle(file_list) # randomly shuffle the list


In [4]:
def prep_dfs(file, tf_df):
    log_df = pd.read_csv(file)
    log_df = log_df.merge(kmean300_df)

    log_df_1 = log_df.loc[log_df['session_position']<=(log_df['session_length']/2)]
    log_df_1['hour_of_day'] = log_df_1['hour_of_day'].astype('float')
    log_df_1['premium'] = log_df_1['premium'].astype('bool')
    log_df_1['weekday'] = log_df_1['date'].astype('datetime64[ns]').dt.dayofweek
    log_df_1 = log_df_1.drop(columns = ['date'])
    log_df_1 = pd.get_dummies(log_df_1, columns=['hist_user_behavior_reason_end', 'hist_user_behavior_reason_start', 'context_type','weekday'], dtype = 'bool')
    log_df_1 = log_df_1.merge(tf_df.drop(columns = ['time_signature','mode','key']))
    
                     
    col_bool = log_df_1.columns[log_df_1.dtypes=='bool']
    col_nonbool = log_df_1.columns[log_df_1.dtypes!='bool'].drop(['session_id','track_id_clean','clus'])
    
    # the non-convertable values will be set to 0
    log_df_1[col_nonbool] = log_df_1[col_nonbool].apply(pd.to_numeric, errors='coerce', downcast = 'float').fillna(0).astype('float32')

    # aggregate the track history where ['skip_2']==True
    log_df_1_summary_skip2True = pd.concat([log_df_1.loc[log_df_1['skip_2']==True].groupby(['session_id'])[col_bool].agg(['mean']), 
                                            log_df_1.loc[log_df_1['skip_2']==True].groupby(['session_id'])[col_nonbool].agg(['mean', 'std', 'median'])],
                                            axis = 1)
    log_df_1_summary_skip2True.columns = log_df_1_summary_skip2True.columns.get_level_values(0)+'_sk2True_'+log_df_1_summary_skip2True.columns.get_level_values(1)
    
    # aggregate the track history where ['skip_2']==False
    log_df_1_summary_skip2False = pd.concat([log_df_1.loc[log_df_1['skip_2']==False].groupby(['session_id'])[col_bool].agg(['mean']), 
                                             log_df_1.loc[log_df_1['skip_2']==False].groupby(['session_id'])[col_nonbool].agg(['mean', 'std', 'median'])],
                                             axis = 1)
    log_df_1_summary_skip2False.columns = log_df_1_summary_skip2False.columns.get_level_values(0)+'_sk2False_'+log_df_1_summary_skip2False.columns.get_level_values(1)
    
    
    log_df_history = log_df_1[['session_id','track_id_clean','skip_2','clus']]


    half_cut = log_df['session_length']/2

    # need to at least include 2 trials, otherwise the log_df_1_summary will confound with all the tracks in the same session

    #1st trial in the 2nd half
    log_df_2_1 = log_df.loc[(log_df['session_position']>half_cut) & (log_df['session_position']<=half_cut+1)]
    log_df_2_1 = log_df_2_1[['session_id','track_id_clean','skip_2','session_position','session_length','clus']]
    log_df_2_1['weight'] = 1

    #2nd trial in the 2nd half
    log_df_2_2 = log_df.loc[(log_df['session_position']>half_cut+1) & (log_df['session_position']<=half_cut+2)]
    log_df_2_2 = log_df_2_2[['session_id','track_id_clean','skip_2','session_position','session_length','clus']]
    log_df_2_2['weight'] = 0.75

    log_df_2 = pd.concat([log_df_2_1,log_df_2_2])
    log_df_2 = log_df_2.merge(log_df_1_summary_skip2True, on='session_id')
    log_df_2 = log_df_2.merge(log_df_1_summary_skip2False, on='session_id')

    sim_file_list = ['../models/SVD/all_tracks/similarity/k300_CanbDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_CosSim.csv',
                     '../models/SVD/all_tracks/similarity/k300_LinCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_ManhDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_HammDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_SpearCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_KendCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_ChebDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_BrayDist.csv']
    score_name_list = ['CanbDist300', 'CosSim300','LinCorr300','ManhDist300','HammDist300','SpearCorr300','KendCorr300','ChebDist','BrayDist']

    return get_sim(log_df_history, log_df_2, sim_file_list, score_name_list)

In [8]:
from timeit import default_timer as timer #to see how long the computation will take

nFile = 0
batch_size = 10
while nFile < 50:
    start = timer()
    nFile += batch_size
    df_lookup_list = []
    for file in file_list[(nFile-batch_size):min(nFile, len(file_list))]:
        df_lookup_list.append(prep_dfs(file, tf_df))

    df_lookup = pd.concat(df_lookup_list)
    df_lookup = df_lookup.merge(tf_df.drop(columns = ['key','time_signature','mode']))
    
    # check whether the column names match with the previous training set
    if nFile>batch_size:
        prev_feature_names = lgb.Booster(model_file='../models/SVD/LightGBM_BayesOpt_dec17/cvbooster'+str(int(nFile-batch_size))+'.txt').feature_name()
        if bool(set(prev_feature_names) - set(df_lookup.columns)): # if there are missing columns
            df_lookup[list(set(prev_feature_names) - set(df_lookup.columns))] = 0 # add the missed columns with 0

    dtrain = lgb.Dataset(df_lookup.drop(columns = ['session_id','track_id_clean','skip_2','weight']).astype('float32'), 
                         label=df_lookup['skip_2'],
                         weight = df_lookup['weight'],
                         free_raw_data=False) # https://lightgbm.readthedocs.io/en/latest/FAQ.html#error-messages-cannot-before-construct-dataset

    def bo_tune_lgb(num_leaves, learning_rate, num_iterations, bagging_fraction, bagging_freq, feature_fraction, min_gain_to_split, nFile, batch_size):
        params = {'num_leaves': int(num_leaves),
                  'learning_rate':learning_rate,
                  'metric': 'binary_error',
                  'num_iterations':int(num_iterations),
                  'bagging_fraction':bagging_fraction,
                  'bagging_freq':int(bagging_freq),
                  'feature_fraction':feature_fraction,
                  'min_gain_to_split':min_gain_to_split,
                  'objective': 'binary',
                  'force_row_wise': True,
                  'num_threads': 5,
                  'verbosity': 0,
                  'tree_learner': 'voting'} #https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html
        if nFile==batch_size:
            init_model=None
        else:
            init_model='../models/SVD/LightGBM_BayesOpt_dec17/cvbooster'+str(int(nFile-batch_size))+'.txt'
            
        try:
            cv_result = lgb.cv(params, dtrain, nfold=4, return_cvbooster=True, init_model=init_model)
        except:
            cv_result = lgb.cv(params, dtrain, nfold=4, return_cvbooster=True, init_model=None)
            
        cv_result['cvbooster'].save_model('../models/SVD/LightGBM_BayesOpt_dec17/cvbooster'+str(int(nFile))+'.txt')
        return 1-cv_result['binary_error-mean'][-1]

    lgb_bo = BayesianOptimization(bo_tune_lgb,
                                  pbounds={'num_leaves': (5, 30),
                                           'learning_rate':(0.001,0.5),
                                           'num_iterations': (100,1000),
                                           'bagging_fraction': (0.8,0.8),
                                           'bagging_freq': (2,2),
                                           'feature_fraction': (0.2, 0.8),
                                           'min_gain_to_split':(0.001,0.1),
                                           'nFile': (nFile,nFile), 
                                           'batch_size': (batch_size,batch_size)
                                            },
                                  random_state=23
                                 )
    # load previous bo logs
    if nFile>batch_size:
        load_logs(lgb_bo, logs=['../models/SVD/LightGBM_BayesOpt_dec17/logs_shuffle'+str(int(nFile-batch_size))+'.json'])


    logger = JSONLogger(path='../models/SVD/LightGBM_BayesOpt_dec17/logs_shuffle'+str(nFile)+'.json')
    lgb_bo.subscribe(Events.OPTIMIZATION_STEP, logger)

    start = timer()
    lgb_bo.maximize(n_iter=15, init_points=10)
    print('Runtime: %0.2fs' % (timer() - start))

KeyboardInterrupt: 